In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import gc

In [2]:
# wget https://kaggle2.blob.core.windows.net/competitions-data/kaggle/3004/test.csv
# wget https://kaggle2.blob.core.windows.net/competitions-data/kaggle/3004/train.csv

In [3]:
csv = pd.read_csv("train.csv")
csv_test = pd.read_csv("test.csv")


In [5]:
csv_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
csv.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def to_one_hot(y):
    return (np.arange(np.unique(y).__len__()) == y[:,None]).astype(float)

In [5]:
X = csv.iloc[:,1:].values.astype(np.float32)
X = np.multiply(X, 1.0 / 255.0)
y = csv.iloc[:,0].values
y_oh = to_one_hot(y)

X_test = csv_test.iloc[:,:].values.astype(np.float32)

In [6]:
print(X.shape, y.shape, y_oh.shape, X_test.shape)

(42000, 784) (42000,) (42000, 10) (28000, 784)


In [7]:
image_size = X.shape[1]
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
print("image_height/image_width = {}/{}".format(image_width, image_height))

image_height/image_width = 28/28


In [8]:
#VALIDATION_SIZE = 2000


# TF graph

In [9]:
BATCH_SIZE = 16

In [10]:
def wb(wshape=[None], bshape=[None], device='/cpu:0'):
    with tf.device(device):
        w = tf.get_variable("w", wshape, initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable('b', bshape, initializer=tf.constant_initializer(0.0))
    return w, b

In [11]:
def conv2d(x, W):
    # W => (patch, patch, depth_from, depth_to)
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [25]:
batch_size = 8
patch_size = 5
depth = 16
num_hidden = 64
train_size = len(X)

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, 28, 28, 1))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 10))
  
  # Variables.
  with tf.variable_scope("convNetConvLayer1"):
        layer1_weights, layer1_biases = wb([3, 3, 1, 32], [32])
  with tf.variable_scope("convNetConvLayer2"):
        layer2_weights, layer2_biases = wb([3, 3, 32, 32], [32])
  with tf.variable_scope("convNetConvLayer3"):
        layer3_weights, layer3_biases = wb([3, 3, 32, 64], [64])
  with tf.variable_scope("convNetConvLayer4"):
        layer4_weights, layer4_biases = wb([3, 3, 64, 64], [64])
#   with tf.variable_scope("convNetConvLayer5"):
#         layer5_weights, layer5_biases = wb([3, 3, 64, 128], [128])
#   with tf.variable_scope("convNetConvLayer6"):
#         layer6_weights, layer6_biases = wb([3, 3, 128, 128], [128])
  with tf.variable_scope("convNetFC1"):
        layer_fc1_weights, layer_fc1_biases = wb([3136, 1024], [1024])
  with tf.variable_scope("convNetFC2"):
        layer_fc2_weights, layer_fc2_biases = wb([1024, 10], [10])
  
  # Model with max_pool.
  def model(data, train=True):
    conv1 = tf.nn.relu(conv2d(data, layer1_weights) + layer1_biases)
    conv2 = tf.nn.relu(conv2d(conv1, layer2_weights) + layer2_biases)
    if train:
        conv2 = tf.nn.dropout(conv2, 0.5)
    pool1 = max_pool_2x2(conv2)
    print("pool1", pool1.get_shape())
    
    conv3 = tf.nn.relu(conv2d(pool1, layer3_weights) + layer3_biases)
    conv4 = tf.nn.relu(conv2d(conv3, layer4_weights) + layer4_biases)
    if train:
        conv4 = tf.nn.dropout(conv4, 0.5)
    pool2 = max_pool_2x2(conv4)
    print("pool2", pool2.get_shape())
    
#     conv5 = tf.nn.relu(conv2d(pool2, layer5_weights) + layer5_biases)
#     conv6 = tf.nn.relu(conv2d(conv5, layer6_weights) + layer6_biases)
#     if train:
#         tf.nn.dropout(conv4, 0.5)
#     pool3 = max_pool_2x2(conv6)
#     print("pool3", pool3.get_shape())
    
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    
    fc1 = tf.nn.relu(tf.matmul(reshape, layer_fc1_weights) + layer_fc1_biases)
    if train:
        fc1 = tf.nn.dropout(fc1, 0.5)
    fc2 = tf.matmul(fc1, layer_fc2_weights) + layer_fc2_biases
    return fc2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  learn_rate  = tf.train.exponential_decay(1e-4, global_step*batch_size, train_size, 0.5, staircase=True)
  optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  with tf.device("/cpu:0"):  
        tf_test_dataset = tf.constant(X_test)
        test_prediction = tf.nn.softmax(model(tf.reshape(tf_test_dataset, (-1, 28, 28, 1)), train=False))
    
gc.collect()

pool1 (8, 14, 14, 32)
pool2 (8, 7, 7, 64)
pool1 (28000, 14, 14, 32)
pool2 (28000, 7, 7, 64)


0

In [13]:

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [26]:
steps = train_size//batch_size * 10
print(steps)
ans = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for i in range(steps):
        t = time.time()
        offset = (i * batch_size) % (train_size - batch_size)
        batch_data = X[offset:(offset + batch_size), :]
        batch_labels = y_oh[offset:(offset + batch_size), :]
        feed_dict = {
            tf_train_dataset : batch_data.reshape((-1, 28, 28, 1)), 
            tf_train_labels : batch_labels
        }

        _, l, pred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (i < 100 and i % 10 == 0) or i % 100 == 0:
            print( "step: {}, accuracy: {}, loss: {}, took {:.03f}".format(i, accuracy(pred,batch_labels ), l, time.time()-t))
    ans = test_prediction.eval()

52500
step: 0, accuracy: 0.0, loss: 16.551673889160156, took 0.167
step: 10, accuracy: 0.0, loss: 8.987409591674805, took 0.025
step: 20, accuracy: 0.0, loss: 5.722247123718262, took 0.031
step: 30, accuracy: 0.0, loss: 7.509032726287842, took 0.029
step: 40, accuracy: 37.5, loss: 3.1486573219299316, took 0.031
step: 50, accuracy: 0.0, loss: 4.42907190322876, took 0.024
step: 60, accuracy: 25.0, loss: 5.88735294342041, took 0.024
step: 70, accuracy: 0.0, loss: 5.055665016174316, took 0.024
step: 80, accuracy: 0.0, loss: 4.0709662437438965, took 0.024
step: 90, accuracy: 0.0, loss: 4.454544544219971, took 0.025
step: 100, accuracy: 37.5, loss: 2.9129772186279297, took 0.024
step: 200, accuracy: 37.5, loss: 2.075410842895508, took 0.021
step: 300, accuracy: 25.0, loss: 2.3625922203063965, took 0.021
step: 400, accuracy: 62.5, loss: 0.8393750786781311, took 0.021
step: 500, accuracy: 75.0, loss: 1.0615410804748535, took 0.021
step: 600, accuracy: 87.5, loss: 0.4297986626625061, took 0.021

In [27]:
ans = np.argmax(ans, 1)

In [28]:
ans.shape

(28000,)

In [29]:
import csv
with open('answer.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["ImageId","Label"])
    for i, a in enumerate(ans):
        writer.writerow([i+1,a])